# Function Calling czyli generowania ustrukturyzowanych danych

documentation : https://platform.openai.com/docs/guides/function-calling

- jako dodatkowy parametr zapytania przekazujemy listę funkcji w postaci nazwy, opisu oraz zestaw parametrów
- poza tym przesyłamy także listę wiadomości, podobnie jak w klasycznej interakcji z modelem
- model w odpowiedzi zwraca nam nazwę funkcji oraz listę wartości jej parametrów
- opcjonalnie (ale jest to zwykle potrzebne) wskazujemy, która funkcja ma być wybrana jako domyślna


make an actual call to OpenAI and apply an “illusionary” function called “get_answer_for_user_query” by guiding AI to return JSON output
function is  `illusionary` because it’s control execution is automatically inferred by OpenAI model using name, description, and output schema but not defined by the developer.

## Example
[make.com](https://medium.com/dev-bits/a-clear-guide-to-openai-function-calling-with-python-dcbc200c5d70#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6IjBlNzJkYTFkZjUwMWNhNmY3NTZiZjEwM2ZkN2M3MjAyOTQ3NzI1MDYiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJhenAiOiIyMTYyOTYwMzU4MzQtazFrNnFlMDYwczJ0cDJhMmphbTRsamRjbXMwMHN0dGcuYXBwcy5nb29nbGV1c2VyY29udGVudC5jb20iLCJhdWQiOiIyMTYyOTYwMzU4MzQtazFrNnFlMDYwczJ0cDJhMmphbTRsamRjbXMwMHN0dGcuYXBwcy5nb29nbGV1c2VyY29udGVudC5jb20iLCJzdWIiOiIxMTQ4NzE3NjUwMjg0OTAzOTczMTkiLCJlbWFpbCI6InJhcGN6eW5za2lAZ21haWwuY29tIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsIm5iZiI6MTcwMDkyNjU4NywibmFtZSI6IkFydHVyIHJhcGN6ecWEc2tpIiwicGljdHVyZSI6Imh0dHBzOi8vbGgzLmdvb2dsZXVzZXJjb250ZW50LmNvbS9hL0FDZzhvY0pycmRKNzh1R0dyMmlMZ3Z5bVZma0tNU3hET3dDeThSQXlBUHpheDEtbz1zOTYtYyIsImdpdmVuX25hbWUiOiJBcnR1ciIsImZhbWlseV9uYW1lIjoicmFwY3p5xYRza2kiLCJsb2NhbGUiOiJlbiIsImlhdCI6MTcwMDkyNjg4NywiZXhwIjoxNzAwOTMwNDg3LCJqdGkiOiJjOTRkYTY1YWUwZWI4OTQ3M2ZlOWU3NmUwMjQ4MmFjOTJhNGM1YjBlIn0.UfqNI5JJcEwc48p8D189cMchHcoyy_inWyfzDHM0W1znomF_us55lmh7i3uEuYejtRFqmEuPXkqUQ_3OzlDxijJIOwIXbJOdvJWaAtoBWEpRqzmT0ipFfID3-WUUMDdiuwt-iTMHyvkkAyoDGTpJ28XDgY9LQkwUvjpcd_SfGB6JrlJDBJtYCXa3sfbyRyh5NLF69NJmdIHCJg1ztPlSWRGKKQvJL8lmBy3B1u4abm1XRwcH1eCcTtmOE_S6Ed3u2tThz-Tfb0LBMBJJszZk_MAPVNsofjfSQ2e93JlEflBmPhmmuoglNGO3OkeBjVL8iVWs3m4z3zX24raudl9KpQ)
first we create schema : 
PyDantic class to structure a model and convert it to JSON schema to avoid verbosity and errors.

goal of this excercise is to return structure answer to question: 
“Have a title and series of steps for a requested question, and generate a JSON-parsable output”.



#### Firt load env variables to get open ai key: 

In [ ]:
from __future__ import absolute_import
from typing import List
from pydantic import BaseModel
import json
from openai import OpenAI
import dotenv
from os import environ
env_file = './.env'
dotenv.load_dotenv(env_file, override=True)
print('MY_VAR = ', environ.get('MY_VAR'))



#### This is most basic example of function calling 

In [ ]:

class SimplePersonModel(BaseModel):
    paragraph: str

simple_schema = SimplePersonModel.model_json_schema()
#print(json.dumps(simple_schema, indent=2))

client = OpenAI(api_key=environ.get('OPENAI_API_KEY'))

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "user", "content": "Write short paragraph about pirates"}
    ],
    functions=[
      {
          "name" : "provide_answer",
          "description" : "Get user answer",
          "parameters" : SimplePersonModel.model_json_schema()
      }
    ],
    function_call={"name": "provide_answer"}
)

#respone for list 
print(response.choices[0].message)
output = json.loads(response.choices[0].message.function_call.arguments)
print(json.dumps(output, indent=2))


#### This is more complex example with list 

In [ ]:

class SimplePersonModel(BaseModel):
    paragraph: str

simple_schema = SimplePersonModel.model_json_schema()
#print(json.dumps(simple_schema, indent=2))

client = OpenAI(api_key=environ.get('OPENAI_API_KEY'))

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "user", "content": "Write short paragraph about pirates"}
    ],
    functions=[
      {
          "name" : "provide_answer",
          "description" : "Get user answer",
          "parameters" : SimplePersonModel.model_json_schema()
      }
    ],
    function_call={"name": "provide_answer"}
)

#respone for list 
print(response.choices[0].message)
output = json.loads(response.choices[0].message.function_call.arguments)
print(json.dumps(output, indent=2))


#### This is example how clasification can be done with function calling 

In [ ]:
from typing import List
import pydantic 
import json
import enum

class TypeEnum(enum.Enum):
    MEMO = "memory"
    NOTE = "notes"
    LINK = "links"
    TODO = "todo"
  

class ClasificationModel(BaseModel):
    command: bool 
    type: TypeEnum = pydantic.Field(...)
    tags : List[str]

list_schema = ClasificationModel.model_json_schema() 

# print(json.dumps(list_schema, indent=2))


def call_ai(): 
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
        {"role": "user", "content": "write report for monday "}
        ],
        functions=[
            {
            "name": "clasify_answer",
            "description": "Describe users query with semantic tags and classify with type",
            "parameters": list_schema 
            }
        ],
        function_call={"name": "clasify_answer"}
    )

    output = json.loads(response.choices[0].message.function_call.arguments)
    print(json.dumps(output, indent=2))

call_ai()

#### This is more complex example where we provide many functions and ai chooses most suiting

In [13]:
from typing import List
from pydantic import BaseModel
import json


class AddSchema(BaseModel):
    first: int
    second: int 

class SubtractSchema(BaseModel):
    first: int
    second: int 

add_schema = AddSchema.model_json_schema() 
subtract_schema = SubtractSchema.model_json_schema() 


# print(json.dumps(list_schema, indent=2))

# Create two possible functions to choose from: 
function_list = [
    {
        "name": "add",
        "description": "add two numbers",
        "parameters": add_schema ,
            "required": [
                "first", "second"
            ]
    },
    {
        "name": "subtract",
        "description": "substract two numbers",
        "parameters": subtract_schema,
            "required": [
                "first", "second"
            ]
    },
]

# call to open ai with those functions 2 and let it deceide which to choose from base on input 

def call_ai(): 
    client = OpenAI(api_key=environ.get('OPENAI_API_KEY'))
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
        {"role": "user", "content": "2 minus 1"}
        ],
        functions=function_list,
    )

    # fetch data from output 
    arguments_json_object = json.loads(response.choices[0].message.function_call.arguments)
    first_number = arguments_json_object['first']
    second_number = arguments_json_object['second']
    function_name = response.choices[0].message.function_call.name

    print(f"Selected function => {function_name}")
    print(f"Output from ai => {json.dumps(arguments_json_object, indent=2)})")


    # Example how to use output
    tools = {
    'add': lambda a, b: a + b,
    'subtract': lambda a, b: a - b,
    }

    if first_number != None and second_number != None and function_name != None: 
        result = tools[function_name](first_number, second_number)
        print(f"result is {result}")


call_ai()

Selected function => subtract
Output from ai => {
  "first": 2,
  "second": 1
})


TypeError: unsupported operand type(s) for &: 'NoneType' and 'int'